In [ ]:
import random
import pprint
import time

# Códigos de color ANSI
LIGHT_BLUE = "\033[94m"  # Azul claro
DARK_BLUE = "\033[34m"   # Azul oscuro
RED = "\033[91m"         # Rojo
GREEN = "\033[92m"       # Verde
RESET = "\033[0m"        # Reset (para volver al color normal)

def menu_inicio():
    '''
        Muestra el menú de inicio del juego, que permite ir al juego directamente, ver las instrucciones o introducir un nombre para el jugador
    '''
    print('¡Bienvenido al Hundir la flota!')
    opcion = None
    nombre_jugador = 'Jugador'
    while opcion != '1':
        opcion = input('Selecciona una opción:\n'
                    '1. Jugar\n '
                    '2. Introducir nombre jugador\n'
                    '3. Ver instrucciones')
        if opcion == '2':
            nombre_jugador = input('Introduce el nombre del jugador: ')
            print(f'Jugador: {nombre_jugador}')
        elif opcion == '3':
            print("Instrucciones del juego de Hundir la flota:\n"
                "- Tablero: El juego se juega en un tablero de 10x10.\n"
                "- Barcos:\n"
                "  * 4 barcos de 1 casilla\n"
                "  * 3 barcos de 2 casillas\n"
                "  * 2 barcos de 3 casillas\n"
                "  * 1 barco de 4 casillas\n"
                "- Colocación de barcos: Los barcos de ambos jugadores (tú y la máquina) se colocan aleatoriamente en el tablero.\n"
                "- Objetivo: Gana el primero que logre hundir todos los barcos del oponente, es decir, el que haga 20 impactos.\n"
                "- Disparo: Para atacar, introduce la fila y columna donde quieres disparar.\n"
                "- Turnos:\n"
                "  * Si aciertas y das en un barco, repetirás tu turno.\n"
                "  * Si fallas, será el turno de la máquina.\n"
                "\n¡Buena suerte!")
        elif opcion == '1':
            break
        else:
            print('No existe esa opción')
    return nombre_jugador


def crea_tablero(casillas = 10):
    '''
        Crea un tablero vacío del número de filas indicado (por defecto 10)
    '''
    fila = ['_' for _ in range(casillas)]
    tablero =[fila.copy() for _ in range(casillas)]  
    return tablero

def espacio_libre(tablero, fila, columna, tamaño, direccion):
    '''
        Verifica si se puede colocar un barco en la posicion determinada, o si esa posición ya está ocupada por otro barco
    '''
    if direccion == 'H':                                # Si la dirección es H, comprueba si el barco se puede colocar desde esa casilla hacia la derecha
        for i in range(tamaño):
            if tablero[fila][columna + i] == 'B':
                espacio_libre = False
                break
            else:
                espacio_libre = True
    if direccion == 'V':                                # Si la dirección es V, comprueba si el barco se puede colocar desde esa casilla hacia abajo
        for i in range(tamaño):
            if tablero[fila + i][columna] == 'B':
                espacio_libre = False
                break
            else:
                espacio_libre = True
    return espacio_libre

def colocar_barco(tablero, tamaño):
    '''
        Coloca un barco en la posición del tablero indicada, siempre que la funcion espacio_libre() sea True. Elige aleatoriamente unas coordenadas y una dirección.
    '''
    barco_colocado = False
    while barco_colocado == False:
        direccion = random.choice(['H', 'V'])
        if direccion == 'H':
            fila = random.randint(0, len(tablero) - 1)
            columna = random.randint(0, (len(tablero)) - tamaño)
            if espacio_libre(tablero,fila,columna,tamaño, direccion) == True:
                for i in range(tamaño):
                    tablero[fila][columna + i] = 'B'
                    barco_colocado = True
        if direccion == 'V':
            fila = random.randint(0, len(tablero) - tamaño)
            columna = random.randint(0, len(tablero) - 1)
            if espacio_libre(tablero,fila,columna,tamaño, direccion) == True:
                for i in range(tamaño):
                    tablero[fila + i][columna] = 'B'
                    barco_colocado = True

def entrada_numero(num_text, entrada_correcta):
    '''
    Asegura que se introduce un número
    '''
    num = None
    try:
        num = int(num_text)
        entrada_correcta = True
    except Exception as e:
        print('Entrada incorrecta. El valor debe ser un número. Prueba otra vez')
        entrada_correcta = False
    return num, entrada_correcta

def disparo_jugador(tablero, disparos, puntos):
    '''
        Realiza el diparo en la posición que el jugador indique, y actualiza los tableros y marcadores
    '''
    cambio_turno = False
    while not cambio_turno:
        pprint.pprint(disparos)

        x_valida = False                                                      # Solicita coordenadas del disparo y asegura que sean números. Si no, vuelve a solicitarlas
        while not x_valida:
            x_text = input('Introduce la fila del disparo: ')
            x, x_valida = entrada_numero(x_text,x_valida)
        y_valida = False
        while not y_valida:
            y_text = input('Introduce la columna del disparo: ')
            y, y_valida = entrada_numero(y_text, y_valida)

        if x not in range(len(tablero)) or y not in range(len(tablero)):      # Comprueba si el disparo está dentro del tablero. Si está fuera, pierdes el turno
            print('Disparo fuera del tablero. Pierdes el turno')
            cambio_turno = True
        elif disparos[x][y] in ['X', 'O']:                                    # Comprueba si has disparado en un sitio repetido. Si es así, se repite tu turno
            print('Ya has disparado ahí. Prueba otra vez')
        elif tablero[x][y] == '_':                                            # Comprueba si el disparo ha dado en el agua
            disparos[x][y] = 'O'
            print('Disparo fallado')
            cambio_turno = True
        elif tablero[x][y] == 'B':                                            # Comprueba si el disparo ha dado en un barco. Si es true, repite turno
            disparos[x][y] = 'X'
            print('¡Buen disparo! Has acertado en un barco')
            puntos = puntos + 1
            cambio_turno = False
            if puntos == 20:                                                  # Si el jugador lleva 20 aciertos, ya no se repite su turno porque se acaba el juego
                cambio_turno = True
    return puntos

def disparo_maquina(tablero, puntos):
    '''
        Realiza el diparo en la posición que la máquina elija aleatoriamente, y actualiza los tableros y marcadores
    '''
    cambio_turno = False
    while not cambio_turno:
        time.sleep(1)
        x = random.randint(0, len(tablero) - 1)    # Asegura que el disparo de la máquina esté dentro de los límites del tablero
        y = random.randint(0, len(tablero) - 1)
        if tablero[x][y] == '_':
            tablero[x][y] = 'O'
            print('Disparo fallado')
            pprint.pprint(tablero)
            cambio_turno = True
        elif tablero[x][y] == 'B':
            tablero[x][y] = f'X'
            print('¡Buen disparo! La máquina ha acertado en un barco')
            pprint.pprint(tablero)
            puntos = puntos + 1
            cambio_turno = False
            if puntos == 20:
                cambio_turno = True
        else:
            continue     # Si la máquina dispara en un punto en el que ya disparó anteriormente, se repite su turno
    return puntos

def mostrar_marcador(puntos_jugador, puntos_maquina, nombre_jugador):
    ''' 
        Muestra el marcador del juego
    '''
    print(f'Puntos {nombre_jugador}: {puntos_jugador}')
    print(f'Puntos Computer: {puntos_maquina}')

            


# Aquí empieza el programa principal
nombre_jugador = menu_inicio()

# Se crean los tableros
tablero_jugador = crea_tablero()
tablero_maquina = crea_tablero()
disparos_jugador = crea_tablero()

# Se colocan aleatoriamente los barcos en el tablero de la máquina
colocar_barco(tablero_maquina, 4)
for i in range(2):
    colocar_barco(tablero_maquina, 3)
for i in range(3):
    colocar_barco(tablero_maquina, 2)
for i in range(4):
    colocar_barco(tablero_maquina, 1) 
pprint.pprint(tablero_maquina)   # en el juego real se eliminaría esta línea para que no muestre en pantalla el tablero de la máquina

# Se colocan aleatoriamente los barcos en el tablero del jugador
colocar_barco(tablero_jugador, 4)
for i in range(2):
    colocar_barco(tablero_jugador, 3)
for i in range(3):
    colocar_barco(tablero_jugador, 2)
for i in range(4):
    colocar_barco(tablero_jugador, 1) 
print('El tablero con tus barcos:')  
pprint.pprint(tablero_jugador)

# Se inician a 0 los marcadores de los jugadores
puntos_jugador = 0
puntos_maquina = 0

# Aquí comienza el bucle principal del juego
fin_juego = False
ronda = 1
while not fin_juego:
    time.sleep(1.5)
    print()
    print(f'RONDA {ronda}')
    mostrar_marcador(puntos_jugador, puntos_maquina, nombre_jugador)
    print()

    # Turno del jugador
    print('Tu turno:')
    puntos_jugador = disparo_jugador(tablero_maquina, disparos_jugador, puntos_jugador)
    print()
    if puntos_jugador == 20:
        print('¡¡ WIN !! Has hundido todos los barcos del enemigo')
        print('Resultado final:')
        mostrar_marcador(puntos_jugador, puntos_maquina, nombre_jugador)
        fin_juego = True
        break

    # Turno de la máquina
    print('Turno de la máquina:')
    puntos_maquina = disparo_maquina(tablero_jugador, puntos_maquina)
    if puntos_maquina == 20:
        print('¡¡ GAME OVER !! Todos tus barcos están hundidos')
        print('Resultado final:')
        mostrar_marcador(puntos_jugador, puntos_maquina, nombre_jugador)
        fin_juego = True
        break

    ronda = ronda + 1





¡Bienvenido al Hundir la flota!
Jugador: Esteban
Instrucciones del juego de Hundir la flota:
- Tablero: El juego se juega en un tablero de 10x10.
- Barcos:
  * 4 barcos de 1 casilla
  * 3 barcos de 2 casillas
  * 2 barcos de 3 casillas
  * 1 barco de 4 casillas
- Colocación de barcos: Los barcos de ambos jugadores (tú y la máquina) se colocan aleatoriamente en el tablero.
- Objetivo: Gana el primero que logre hundir todos los barcos del oponente, es decir, el que haga 20 impactos.
- Disparo: Para atacar, introduce la fila y columna donde quieres disparar.
- Turnos:
  * Si aciertas y das en un barco, repetirás tu turno.
  * Si fallas, será el turno de la máquina.

¡Buena suerte!
[['_', '_', 'B', '_', '_', '_', '_', '_', 'B', '_'],
 ['_', '_', '_', '_', '_', 'B', '_', '_', 'B', '_'],
 ['_', '_', '_', '_', '_', 'B', '_', '_', 'B', '_'],
 ['_', '_', '_', '_', '_', '_', '_', '_', 'B', '_'],
 ['_', '_', 'B', '_', '_', '_', '_', '_', 'B', '_'],
 ['_', '_', 'B', '_', '_', '_', '_', '_', 'B', '

In [30]:
fil =['-','-','-','-','-','-']

tamano = 5
for i in range(tamano):
    fil[i] = 'X'

fil
len(fil)

6

In [ ]:
# Otra forma para contar los puntos
puntos = 0
for i in range(len(tablero)):
    puntos = puntos + tablero[i].count('X')